In [1]:
import pandas as pd

In [2]:
tweet = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [6]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

There are 7613 rows and 5 columns in train
There are 3263 rows and 4 columns in test


In [9]:
df=pd.concat([tweet,test])
tweet.shape
#test.shape
#df.shape

(7613, 5)